# Tree-Search for Heads-Up No Limit Holdem
 The goal of this notebook is to create and test a montecarlo-simulation-algorithm to beat given policies in ho limit holdem heads-up

In [1]:
from game.PokerGame import HUPoker
from Agent2 import Agent2

2022-06-11 12:20:33.788225: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-06-11 12:20:33.788256: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2022-06-11 12:20:35.581490: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2022-06-11 12:20:35.581522: W tensorflow/stream_executor/cuda/cuda_driver.cc:269] failed call to cuInit: UNKNOWN ERROR (303)
2022-06-11 12:20:35.581542: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (cromi-Lenovo-V15-ADA): /proc/driver/nvidia/version does not exist
2022-06-11 12:20:35.581758: I tensorflow/core/platform/c

In [2]:
lr = 0.001
gamma = 1
n_actions = 12
epsilon = 1
batch_size = 32
input_dims = (102,)
stacksize = 10
fname = 'trainedModels/PokerAI10BB.h5'
layerSizes = (256,128,64, 32, 16)
newAgent = Agent2(lr, gamma, n_actions, 0.2, batch_size, input_dims, stacksize, fname=fname, handEval=True,
                  layerSizes=layerSizes, mem_size=100000)
oldAgent = Agent2(lr, gamma, n_actions, 0, batch_size, input_dims, stacksize, fname=fname, handEval=True,
                  layerSizes=layerSizes, mem_size=0)
newAgent.load_model()
oldAgent.load_model()
stack_hero = stacksize
stack_villain = stacksize
agentHero = newAgent
agentVillain = oldAgent
game = HUPoker(stack_hero, stack_villain, agentHero, agentVillain)

In [3]:
import numpy as np

n_rounds = 1001
score = 0
for i in range(n_rounds):

    done =True

    while done:
        observation, done = game.reset()

    while not done:

        error = False
        action = agentHero.choose_action(observation)
        observation = np.append(observation,[0, 0, 0, 0, 0])

        try:
            done = game.step(action) #sometimes the chosen actions lead to too many moves on a street (over 20) This happens when players both chose to engage in minraise battles.
            #we want to severely punish this behavior.
        except IndexError: #error happens when more than 20 actions are attepted on a single street.
            done = True
            error = True

        if not done:
            reward = 0
        else:
            if error:
                reward = -stacksize
            else:
                reward = 0.5+game.stack_sb - game.starting_hero if game.position == 0 else 1+game.stack_bb - game.starting_hero #to take into account the blinds are posted anyway regardless of action.
        score += reward - 0.5 if game.position == 0 else reward - 1
        avgScore = round(score//(i+0.00001),2)
        observation = game.observation
        print(i)


0
0
0
0
1
1
1
1
2
2
2
3
3
3
3
4
4


KeyboardInterrupt: 

In [4]:
import copy
class MCSim:
    def __init__(self,gamestate, range_vil, policy_network):
        self.game = copy.deepcopy(gamestate)
        self.range = range_vil
        self.network = policy_network



In [6]:
values = list(range(2,15))
suits = [0, 1, 2, 3]
deck = []
for value in values:
    for suit in suits:
        deck.append((value, suit))
tree = MCSim(1, [], newAgent.q_eval)

In [13]:
from itertools import combinations
import numpy as np
combos = combinations(deck,2)
np.array(list(combos))

array([[[ 2,  0],
        [ 2,  1]],

       [[ 2,  0],
        [ 2,  2]],

       [[ 2,  0],
        [ 2,  3]],

       ...,

       [[14,  1],
        [14,  2]],

       [[14,  1],
        [14,  3]],

       [[14,  2],
        [14,  3]]])